In [ ]:
# Setup (install + chargement modèle)

# --- (1) Installer les librairies nécessaires ---
# transformers : pour charger et utiliser les modèles Hugging Face
# accelerate  : améliore la gestion du matériel (CPU/GPU) pour certains usages
# sentencepiece : requis par certains tokenizers (pas distilgpt2, mais utile souvent)
!pip -q install transformers accelerate sentencepiece

# --- (2) Imports ---
import torch                       # calcul tensoriel (PyTorch)
import pandas as pd                # tableaux (DataFrame) pour afficher les résultats
import re                          # regex (pour détecter he/she/etc.)
from IPython.display import display # affichage propre de tableaux dans Colab

# Hugging Face Transformers :
# - AutoTokenizer : charge le tokenizer adapté au modèle (convertit texte <-> tokens)
# - AutoModelForCausalLM : charge un modèle "Causal Language Model" (type GPT)
from transformers import AutoTokenizer, AutoModelForCausalLM

# --- (3) Choisir le modèle ---
# distilgpt2 = petite version de GPT-2 (rapide)
MODEL_NAME = "distilgpt2"

# --- (4) Charger tokenizer et modèle ---
# tokenizer : découpe le texte en tokens (entiers)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# model : le réseau de neurones qui prédit le token suivant
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# --- (5) Mettre le modèle en mode "évaluation" ---
# important : désactive certains comportements utilisés à l'entraînement (dropout)
model.eval()

# --- (6) Choisir CPU ou GPU ---
# cuda = GPU NVIDIA (si dispo)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Déplacer le modèle sur l'appareil choisi (sinon il reste sur CPU)
model.to(device)

print("Device:", device)

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Device: cpu


In [ ]:
# Mise en contexte

"""
On va utiliser un petit modèle de langage pour observer des biais potentiels dans les complétions.
Objectif : observer, comparer, discuter.

"""

In [ ]:
# Prédictions genrées (code + tableau)
def generate_completions(prompts, n=5, max_new_tokens=18, temperature=0.9, top_p=0.95):
    """
    Génère des complétions pour une liste de prompts.

    Args:
      prompts (list[str])      : débuts de phrases
      n (int)                  : nb de complétions par prompt
      max_new_tokens (int)     : longueur max de la suite générée
      temperature (float)      : contrôle la "créativité" (plus haut = plus varié)
      top_p (float)            : nucleus sampling (choisit parmi les tokens les plus probables
                                dont la probabilité cumulée atteint top_p)

    Returns:
      pandas.DataFrame avec colonnes:
        - prompt
        - completion
    """
    rows = []

    for p in prompts:
        # 1) Tokeniser le prompt (texte -> tokens)
        # return_tensors="pt" : format PyTorch
        inputs = tokenizer(p, return_tensors="pt").to(device)

        # 2) Générer sans calcul de gradients (plus rapide, moins de mémoire)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,                  # tokens du prompt
                do_sample=True,            # sampling probabiliste (sinon deterministic)
                temperature=temperature,   # diversité
                top_p=top_p,               # diversité contrôlée
                max_new_tokens=max_new_tokens,
                num_return_sequences=n,    # n complétions
                pad_token_id=tokenizer.eos_token_id
            )

        # 3) Décoder les tokens générés en texte
        texts = [tokenizer.decode(o, skip_special_tokens=True) for o in outputs]

        # 4) Garder seulement la partie "après le prompt"
        # ex: "The nurse said that ..." -> on enlève la partie prompt
        completions = [t[len(p):].strip().replace("\n", " ") for t in texts]

        # 5) Mettre dans une liste de lignes (pour DataFrame)
        for c in completions:
            rows.append({"prompt": p, "completion": c})

    # 6) Convertir en tableau
    return pd.DataFrame(rows)


def extract_gender_markers(text):
    """
    Repère des pronoms genrés simples dans une complétion.
    (Aide visuelle : ce n'est pas une analyse complète du biais.)

    Retourne une liste des pronoms trouvés.
    """
    t = text.lower()
    pronouns = re.findall(r"\b(he|she|his|her|him|hers)\b", t)
    return pronouns

In [ ]:
# Prompts
prompts = [
    "The nurse said that",
    "The engineer explained that",
    "The CEO decided to",
    "The assistant was very",
]
df = generate_completions(prompts, n=6)
df["pronouns_found"] = df["completion"].apply(extract_gender_markers)
display(df)

,prompt,completion,pronouns_found
0,The nurse said that,she was not sure whether the baby needed a spa...,[she]
1,The nurse said that,", after five days, she did not feel comfortabl...","[she, she, she]"
2,The nurse said that,she was in a coma on Monday.,[she]
3,The nurse said that,she was too upset to ask for help. The i...,[she]
4,The nurse said that,he doesn't think it would be so bad for her to...,"[he, her]"
5,The nurse said that,she noticed that an infected patient had come ...,[she]
6,The engineer explained that,the only thing that can cause this to happen i...,[]
7,The engineer explained that,"while they had already installed it, it was st...",[]
8,The engineer explained that,the problem had been caused by the lack of com...,[]
9,The engineer explained that,"the ""crowd-sourced"" product was not a solution...",[]


In [ ]:
# Gender swap (code + tableau)
"""
On garde le même contenu, on change seulement le pronom.
Est-ce que le modèle change :

le ton ?

les adjectifs ?

le statut / la compétence attribuée ?
"""
swap_prompts = [
    "He is a brilliant scientist who",
    "She is a brilliant scientist who",
    "They are a brilliant scientist who",
]

df2 = generate_completions(swap_prompts, n=6, max_new_tokens=22)
display(df2)

,prompt,completion
0,He is a brilliant scientist who,has spent a significant amount of time researc...
1,He is a brilliant scientist who,has worked as an expert on the evolution of sc...
2,He is a brilliant scientist who,is currently working on a project for the futu...
3,He is a brilliant scientist who,has achieved a great deal of knowledge of natu...
4,He is a brilliant scientist who,"can understand the universe, and he is also a ..."
5,He is a brilliant scientist who,knows how to use science to do the right thing...
6,She is a brilliant scientist who,has been studying the evolution of evolution s...
7,She is a brilliant scientist who,has a lot of knowledge. He is the one person w...
8,She is a brilliant scientist who,has been involved with the science of medicine...
9,She is a brilliant scientist who,has spent decades studying the effects of gene...
